In [16]:
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 32.4 MB/s eta 0:00:00a 0:00:01
  Using cached contourpy-1.3.3-cp311-cp311-macosx_11_0_arm64.whl (270 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 70.3 MB/s eta 0:00:0000:01
  Using cached kiwisolver-1.4.9-cp311-cp311-macosx_11_0_arm64.whl (65 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 26.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [25]:
import yfinance as yf
from datetime import date, timedelta
import pandas as pd


def get_earnings_dates():

    calendars = yf.Calendars()
    start_date = date.today()
    end_date = start_date + timedelta(days=7)

    df = calendars.get_earnings_calendar(market_cap=300000000, limit=100, start=start_date, end=end_date)
    df = df.reset_index()
    df['Earnings Date'] = pd.to_datetime(df['Event Start Date']).dt.date

    return df[['Symbol', 'Company', 'Event Name', 'Earnings Date', 'EPS Estimate', 'Reported EPS', 'Surprise(%)']]

In [26]:
df = get_earnings_dates()

df

,Symbol,Company,Event Name,Earnings Date,EPS Estimate,Reported EPS,Surprise(%)
0,MSFT,Microsoft Corporation,Q2 2026 Earnings Announcement,2026-01-28,3.85,NaN,NaN
1,META,"Meta Platforms, Inc.",Q4 2025 Earnings Announcement,2026-01-28,8.20,NaN,NaN
2,TSLA,"Tesla, Inc.",Q4 2025 Earnings Announcement,2026-01-28,0.45,NaN,NaN
3,LRCX,Lam Research Corporation,Q2 2026 Earnings Announcement,2026-01-28,1.17,NaN,NaN
4,T,AT&T Inc.,Q4 2025 Earnings Announcement,2026-01-28,0.47,NaN,NaN
5,NOW,"ServiceNow, Inc.",Q4 2025 Earnings Announcement,2026-01-28,0.89,NaN,NaN
6,SBUX,Starbucks Corporation,Q1 2026 Earnings Announcement,2026-01-28,0.59,NaN,NaN
7,SLB,SLB N.V.,Q4 2025 Earnings Announcement,2026-01-23,0.74,NaN,NaN
8,ERIC,Telefonaktiebolaget LM Ericsson (publ),Q4 2025 Earnings Announcement,2026-01-23,0.25,NaN,NaN
9,LUV,Southwest Airlines Co.,Q4 2025 Earnings Announcement,2026-01-28,0.57,NaN,NaN


In [19]:
import yfinance as yf
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

def get_fed_futures_analysis(months_out=12):
    # 1. Setup Month Codes and Date
    month_codes = {
        1: 'F', 2: 'G', 3: 'H', 4: 'J', 5: 'K', 6: 'M',
        7: 'N', 8: 'Q', 9: 'U', 10: 'V', 11: 'X', 12: 'Z'
    }
    today = datetime.date.today()
    data_list = []

    print(f"1. Fetching Futures Chain starting {today.strftime('%b %Y')}...")

    # 2. Fetch Data Loop
    for i in range(months_out + 1): # +1 to ensure we get current month (Spot) + 12 months out
        target_date = today + relativedelta(months=+i)
        m_code = month_codes[target_date.month]
        y_str = str(target_date.year)[-2:]
        ticker = f"ZQ{m_code}{y_str}.CBT"

        try:
            # Fetch just the last day's data
            df = yf.Ticker(ticker).history(period="5d")
            
            if not df.empty:
                last_price = df['Close'].iloc[-1]
                implied_rate = 100 - last_price
                
                data_list.append({
                    'Ticker': ticker,
                    'Month': target_date.strftime('%b %Y'),
                    'Price': last_price,
                    'Implied_Rate': implied_rate
                })
        except Exception as e:
            pass # Skip missing contracts

    # 3. Create DataFrame
    df_results = pd.DataFrame(data_list)

    if df_results.empty:
        return pd.DataFrame() # Return empty if failed

    # 4. Define "Spot Rate" (The Implied Rate of the Front-Month Contract)
    # We assume the first item in our list (Current Month) is the current market spot rate
    spot_rate = df_results.iloc[0]['Implied_Rate']
    
    print(f"   > Market Spot Rate (Front-Month Proxy): {spot_rate:.3f}%")

    # 5. Calculate Implied Cuts
    # Formula: (Spot - Future_Rate) / 0.25
    # Positive result = Cuts. Negative result = Hikes.
    df_results['Delta_vs_Spot'] = spot_rate - df_results['Implied_Rate'] 
    df_results['Cuts_Priced_In'] = df_results['Delta_vs_Spot'] / 0.25

    # formatting for readability
    df_results = df_results[['Month', 'Ticker', 'Price', 'Implied_Rate', 'Cuts_Priced_In']]
    
    return df_results, spot_rate

# --- Execution ---
df, current_spot = get_fed_futures_analysis(months_out=12)

1. Fetching Futures Chain starting Jan 2026...
   > Market Spot Rate (Front-Month Proxy): 3.640%


In [21]:
df

,Month,Ticker,Price,Implied_Rate,Cuts_Priced_In
0,Jan 2026,ZQF26.CBT,96.360001,3.639999,0.000000
1,Feb 2026,ZQG26.CBT,96.370003,3.629997,0.040009
2,Mar 2026,ZQH26.CBT,96.385002,3.614998,0.100006
3,Apr 2026,ZQJ26.CBT,96.400002,3.599998,0.160004
4,May 2026,ZQK26.CBT,96.440002,3.559998,0.320007
5,Jun 2026,ZQM26.CBT,96.489998,3.510002,0.519989
6,Jul 2026,ZQN26.CBT,96.555000,3.445000,0.779999
7,Aug 2026,ZQQ26.CBT,96.614998,3.385002,1.019989
8,Sep 2026,ZQU26.CBT,96.654999,3.345001,1.179993
9,Oct 2026,ZQV26.CBT,96.709999,3.290001,1.399994
